# plumbum Tutorial

This notebook walks through the core features of `pdum.plumbum`, starting with synchronous pipelines and progressing to asynchronous iterator workflows.

## Quick Start

Use the `@pb` decorator to turn ordinary functions into operators. Compose them with `|` and execute with `>>`. Partial application is available via `.partial(...)`.

In [ ]:
from pdum.plumbum import pb


@pb
def add(x, amount):
    return x + amount


@pb
def multiply(x, factor):
    return x * factor


(5 >> add.partial(3), 5 >> (add.partial(3) | multiply.partial(2)))

### Partial Application

Accumulate arguments across multiple steps before running the pipeline.

In [ ]:
@pb
def add_three(x, a, b, c):
    return x + a + b + c


op = add_three.partial(1)
op = op.partial(2)
op = op.partial(3)
10 >> op

### Plain Functions as Operators

Plain callables are wrapped automatically when combined with `|`.

In [ ]:
from functools import partial


def plain_increment(x):
    return x + 3


def plain_add(x, n):
    return x + n


pipeline = multiply.partial(2) | plain_increment
pipeline_result = 5 >> pipeline

pipeline_with_extra = multiply.partial(2) | partial(plain_add, n=4)
(5 >> pipeline_with_extra, pipeline_result)

### Data Type Flexibility

Operators make no assumptions about the threaded data.

In [ ]:
"hello" >> pb(str.upper) >> pb(lambda s: s + "!")
{"a": 1} >> pb(lambda d: {**d, "b": 2})


class Point:
    def __init__(self, x, y):
        self.x, self.y = x, y

    def __repr__(self):
        return f"Point({self.x}, {self.y})"


@pb
def translate(point, dx, dy):
    return Point(point.x + dx, point.y + dy)


Point(1, 2) >> translate.partial(5, 3)

### Debugging Pipelines

Insert `pb(print)` to inspect intermediate values.

In [ ]:
result = 10 >> add.partial(5) >> pb(print) >> multiply.partial(2) >> pb(print)

result

## Asynchronous Pipelines

Use `apb` to wrap coroutine functions. The pipeline returns a coroutine that you await.

In [ ]:
import asyncio
from pdum.plumbum import apb


@apb
async def fetch_amount(value):
    await asyncio.sleep(0)
    return value + 5


@apb
async def scale(value, factor):
    await asyncio.sleep(0)
    return value * factor


async def run_async_pipeline():
    pipeline = fetch_amount | scale.partial(2)
    return await (10 >> pipeline)


asyncio.run(run_async_pipeline())

### Mixing Sync and Async Operators

`apb` can wrap synchronous callables so they join async pipelines.

In [ ]:
@apb
async def async_double(x):
    await asyncio.sleep(0)
    return x * 2


@apb
def add_sync(x, delta):
    return x + delta


async def run_mixed_pipeline():
    pipeline = async_double | add_sync.partial(3) | scale.partial(2)
    return await (5 >> pipeline)


asyncio.run(run_mixed_pipeline())

## Asynchronous Iterator Pipelines

Decorate async-iterator operators with `aipb` or use helpers in `pdum.plumbum.aiterops`.

In [ ]:
from pdum.plumbum import aipb
from pdum.plumbum.aiterops import filter as aiter_filter, map as aiter_map


async def source(limit=5):
    for value in range(limit):
        await asyncio.sleep(0)
        yield value


@aipb
async def keep_even(stream):
    async for item in stream:
        if item % 2 == 0:
            yield item


async def run_async_iter_pipeline():
    pipeline = aiter_map(async_double) | keep_even
    results = []
    async for value in await (source() >> pipeline):
        results.append(value)
    return results


asyncio.run(run_async_iter_pipeline())

### Using Async Iterator Helpers

`aiter_map` and `aiter_filter` accept `pb`/`apb` operators or normal callables.

In [ ]:
async def run_async_iter_helpers():
    pipeline = aiter_map(fetch_amount) | aiter_filter(lambda x: x % 3 == 0)
    results = []
    async for value in await (source(6) >> pipeline):
        results.append(value)
    return results


asyncio.run(run_async_iter_helpers())